In [0]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import requests
from sklearn.neighbors import NearestNeighbors

In [0]:
data_state = pd.read_csv('StateData.csv')

change_keys = {}
for key in data_state.keys():
    change_keys[key] = key.lower()
data_state = data_state.rename(columns=change_keys)

data_state = data_state.rename(columns={'year4':'year'})
data_state.head()

,surveyyr,year,yearofdata,id,idchanged,state_code,type_code,county,name,fips_code_state,...,emp_retire_sec_mortgages,emp_retire_sec_misc_inv,emp_retire_sec_oth_nong,unemp_comp_cash___sec,unemp_comp_bal_in_us_trs,unemp_comp_other_balance,nonin_trust_cash___sec,sinking_fd_cash___sec,bond_fd_cash___sec,oth_nonin_fd_cash___sec
0,16,2016,NaN,10000000,NaN,1,0,0,ALABAMA,1,...,1815,0,4143961,529107,529107,0,12913490,1464402,176157,11272931
1,16,2016,NaN,20000000,NaN,2,0,0,ALASKA,2,...,0,1145382,1867671,433220,433220,0,64944768,3662764,153822,61128182
2,16,2016,NaN,30000000,NaN,3,0,0,ARIZONA,4,...,0,6111807,1821970,322017,322017,0,18794165,5809405,429626,12555134
3,16,2016,NaN,40000000,NaN,4,0,0,ARKANSAS,5,...,4142,10936752,3783960,485792,485792,0,8608523,1664378,358939,6585206
4,16,2016,NaN,50000000,NaN,5,0,0,CALIFORNIA,6,...,5927,17058534,103383571,-3088756,11711,-3100467,107409539,29880672,5039254,72489613


In [0]:
def get_missing(df):
  missing = (~df.fillna(0).astype(bool)).sum()/df.shape[0]
  return missing

def get_keys(df, match):
    return [key for key in df.keys() if match.lower() in key.lower()]
  
def drop_keys(df, keys):
    return df.drop(columns=keys)  

In [0]:
print(get_keys(data_state, 'year'))
print(get_keys(data_state, 'yr'))
print(get_keys(data_state, 'id'))
print(get_keys(data_state, 'code'))
print(get_keys(data_state, 'fips'))

['year', 'yearofdata', 'yearpop', 'st_debt_end_of_year']
['surveyyr', 'unemp_payroll_tax']
['id', 'idchanged', 'individual_income_tax', 'local_igr_interschool_aid', 'chg_solid_waste_mgmt', 'tot_assist___subsidies']
['state_code', 'type_code', 'fips_code_state', 'schlevcode', 'functioncode']
['fips_code_state', 'fips_county', 'fips_place']


In [0]:
keys_to_drop = ['yearofdata', 'yearpop', 'surveyyr', 
                'idchanged', 
                'state_code', 'type_code', 'fips_code_state', 'schlevcode', 'functioncode',
                'fips_code_state', 'fips_county', 'fips_place'
               ]
data_state = drop_keys(data_state, keys_to_drop)
data_state.head()

,year,id,county,name,fyenddate,population,enrollment,total_revenue,total_rev_own_sources,general_revenue,...,emp_retire_sec_mortgages,emp_retire_sec_misc_inv,emp_retire_sec_oth_nong,unemp_comp_cash___sec,unemp_comp_bal_in_us_trs,unemp_comp_other_balance,nonin_trust_cash___sec,sinking_fd_cash___sec,bond_fd_cash___sec,oth_nonin_fd_cash___sec
0,2016,10000000,0,ALABAMA,930.0,4858979,NaN,30172490,20650521,25288886,...,1815,0,4143961,529107,529107,0,12913490,1464402,176157,11272931
1,2016,20000000,0,ALASKA,630.0,738432,NaN,8060170,5206968,7628651,...,0,1145382,1867671,433220,433220,0,64944768,3662764,153822,61128182
2,2016,30000000,0,ARIZONA,630.0,6828065,NaN,38241052,23700590,34582961,...,0,6111807,1821970,322017,322017,0,18794165,5809405,429626,12555134
3,2016,40000000,0,ARKANSAS,630.0,2978204,NaN,21431620,14000079,20346096,...,4142,10936752,3783960,485792,485792,0,8608523,1664378,358939,6585206
4,2016,50000000,0,CALIFORNIA,630.0,39144818,NaN,322332307,227996024,282907926,...,5927,17058534,103383571,-3088756,11711,-3100467,107409539,29880672,5039254,72489613


In [0]:
data_state = drop_keys(data_state, ['county', 'fyenddate'])

In [0]:
data_state['id'].nunique()*data_state['year'].nunique(),  data_state.shape[0]

(2448, 2098)

In [0]:
data_state[['id', 'name']].nunique()

id       51
name    102
dtype: int64

In [0]:
data_state[['name', 'id']].drop_duplicates().sort_values(by='name')

,name,id
0,ALABAMA,10000000
510,ALABAMA STATE GOVT,10000000
1,ALASKA,20000000
511,ALASKA STATE GOVT,20000000
2,ARIZONA,30000000
512,ARIZONA STATE GOVT,30000000
3,ARKANSAS,40000000
513,ARKANSAS STATE GOVT,40000000
4,CALIFORNIA,50000000
514,CALIFORNIA STATE GOVT,50000000


In [0]:
names = data_state['name'].drop_duplicates().tolist()
names = [name for name in names if name.endswith('STATE GOVT')]
len(names)

50

In [0]:
names

['ALABAMA STATE GOVT',
 'ALASKA STATE GOVT',
 'ARIZONA STATE GOVT',
 'ARKANSAS STATE GOVT',
 'CALIFORNIA STATE GOVT',
 'COLORADO STATE GOVT',
 'CONNECTICUT STATE GOVT',
 'DELAWARE STATE GOVT',
 'FLORIDA STATE GOVT',
 'GEORGIA STATE GOVT',
 'HAWAII STATE GOVT',
 'IDAHO STATE GOVT',
 'ILLINOIS STATE GOVT',
 'INDIANA STATE GOVT',
 'IOWA STATE GOVT',
 'KANSAS STATE GOVT',
 'KENTUCKY STATE GOVT',
 'LOUISIANA STATE GOVT',
 'MAINE STATE GOVT',
 'MARYLAND STATE GOVT',
 'MASSACHUSETTS STATE GOVT',
 'MICHIGAN STATE GOVT',
 'MINNESOTA STATE GOVT',
 'MISSISSIPPI STATE GOVT',
 'MISSOURI STATE GOVT',
 'MONTANA STATE GOVT',
 'NEBRASKA STATE GOVT',
 'NEVADA STATE GOVT',
 'NEW HAMPSHIRE STATE GOVT',
 'NEW JERSEY STATE GOVT',
 'NEW MEXICO STATE GOVT',
 'NEW YORK STATE GOVT',
 'NORTH CAROLINA STATE GOVT',
 'NORTH DAKOTA STATE GOVT',
 'OHIO STATE GOVT',
 'OKLAHOMA STATE GOVT',
 'OREGON STATE GOVT',
 'PENNSYLVANIA STATE GOVT',
 'RHODE ISLAND STATE GOVT',
 'SOUTH CAROLINA STATE GOVT',
 'SOUTH DAKOTA STATE G

In [0]:
data_state['name'].replace(to_replace=names, value=[name.replace(' STATE GOVT', '') for name in names], inplace=True)
data_state['name'].replace('WASHINGTON DC', 'WASHINGTON DC CITY', inplace=True)

In [0]:
data_state[['id', 'name']].nunique()

id      51
name    51
dtype: int64

In [0]:
data_state['id'].nunique()*data_state['year'].nunique(), data_state.shape[0]

(2448, 2098)

In [0]:
data_state.head()

,year,id,name,population,enrollment,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,...,emp_retire_sec_mortgages,emp_retire_sec_misc_inv,emp_retire_sec_oth_nong,unemp_comp_cash___sec,unemp_comp_bal_in_us_trs,unemp_comp_other_balance,nonin_trust_cash___sec,sinking_fd_cash___sec,bond_fd_cash___sec,oth_nonin_fd_cash___sec
0,2016,10000000,ALABAMA,4858979,NaN,30172490,20650521,25288886,15766917,9919794,...,1815,0,4143961,529107,529107,0,12913490,1464402,176157,11272931
1,2016,20000000,ALASKA,738432,NaN,8060170,5206968,7628651,4775449,1042164,...,0,1145382,1867671,433220,433220,0,64944768,3662764,153822,61128182
2,2016,30000000,ARIZONA,6828065,NaN,38241052,23700590,34582961,20042499,14676375,...,0,6111807,1821970,322017,322017,0,18794165,5809405,429626,12555134
3,2016,40000000,ARKANSAS,2978204,NaN,21431620,14000079,20346096,12914555,9452883,...,4142,10936752,3783960,485792,485792,0,8608523,1664378,358939,6585206
4,2016,50000000,CALIFORNIA,39144818,NaN,322332307,227996024,282907926,188571643,155231252,...,5927,17058534,103383571,-3088756,11711,-3100467,107409539,29880672,5039254,72489613


In [0]:
missing = get_missing(data_state)
missing.describe()

count    581.000000
mean       0.444662
std        0.407490
min        0.000000
25%        0.010963
50%        0.354147
75%        0.924690
max        1.000000
dtype: float64

# Here give it to Lohith. Ask him to fill in whatever he can. And then take good_attributes.

In [0]:
data_state.to_csv('RECleaned_StateData1.csv', index=False)

In [0]:
a = pd.read_csv('RECleaned_StateData1.csv')
a.head()

,year,id,name,population,enrollment,total_revenue,total_rev_own_sources,general_revenue,gen_rev_own_sources,total_taxes,...,emp_retire_sec_mortgages,emp_retire_sec_misc_inv,emp_retire_sec_oth_nong,unemp_comp_cash___sec,unemp_comp_bal_in_us_trs,unemp_comp_other_balance,nonin_trust_cash___sec,sinking_fd_cash___sec,bond_fd_cash___sec,oth_nonin_fd_cash___sec
0,2016,10000000,ALABAMA,4858979,NaN,30172490,20650521,25288886,15766917,9919794,...,1815,0,4143961,529107,529107,0,12913490,1464402,176157,11272931
1,2016,20000000,ALASKA,738432,NaN,8060170,5206968,7628651,4775449,1042164,...,0,1145382,1867671,433220,433220,0,64944768,3662764,153822,61128182
2,2016,30000000,ARIZONA,6828065,NaN,38241052,23700590,34582961,20042499,14676375,...,0,6111807,1821970,322017,322017,0,18794165,5809405,429626,12555134
3,2016,40000000,ARKANSAS,2978204,NaN,21431620,14000079,20346096,12914555,9452883,...,4142,10936752,3783960,485792,485792,0,8608523,1664378,358939,6585206
4,2016,50000000,CALIFORNIA,39144818,NaN,322332307,227996024,282907926,188571643,155231252,...,5927,17058534,103383571,-3088756,11711,-3100467,107409539,29880672,5039254,72489613


In [0]:
good_attributes = missing[missing<.5].index.tolist()
print(len(good_attributes))
good_attributes[:5]

330


['year', 'id', 'name', 'population', 'total_revenue']

In [0]:
good_attributes = missing[missing<.1].index.tolist()
print(len(good_attributes))
good_attributes[:5]

223


['year', 'id', 'name', 'population', 'total_revenue']

In [0]:
supported_attributes = [
  "Total_Revenue",
  "Total_Rev_Own_Sources",
  "Total_Taxes",
  "Total_Gen_Sales_Tax",
  "Total_Select_Sales_Tax",
  "Total_License_Taxes",
  "Motor_Vehicle_License_Total",
  "Total_Income_Taxes",
  "Total_IG_Revenue",
  "Total_Fed_IG_Revenue",
  "Total_State_IG_Revenue",
  "Total_General_Charges",
  "Chg_Total_Education",
  "Chg_Total_Elem_Education",
  "Chg_Total_High_Ed",
  "Chg_Total_Nat_Res",
  "Prop_Sale_Total",
  "Total_Utility_Revenue",
  "Total_Insur_Trust_Rev",
  "Total_Insur_Trust_Ctrb",
  "Total_Emp_Ret_Rev",
  "Emp_Ret_Total_Ctrib",
  "Total_Unemp_Rev",
  "Total_Expenditure",
  "Total_IG_Expenditure",
  "Total_Current_Expend",
  "Total_Current_Oper",
  "Total_Capital_Outlays",
  "Total_Construction",
  "Total_Other_Capital_Outlays",
  "Total_Interest_on_Debt",
  "Total_Insur_Trust_Ben",
  "Total_Salaries___Wages",
  "Air_Trans_Total_Expend",
  "Correct_Total_Exp",
  "Total_Educ_Total_Exp",
  "Total_Educ_Direct_Exp",
  "Total_Educ_Assist___Sub",
  "Total_Educ_Cap_Outlay",
  "Total_Educ_Current_Exp",
  "Total_Educ_Construct",
  "Elem_Educ_Total_Exp",
  "Higher_Ed_Total_Exp",
  "Educ_NEC_Total_Expend",
  "Fin_Admin_Total_Exp",
  "Fire_Prot_Total_Expend",
  "Judicial_Total_Expend",
  "Cen_Staff_Total_Expend",
  "Gen_Pub_Bldg_Total_Exp",
  "Health_Total_Expend",
  "Total_Hospital_Total_Exp",
  "Total_Hospital_Dir_Exp",
  "Total_Hospital_Cap_Out",
  "Total_Hospital_Current_Exp",
  "Total_Hospital_Construct",
  "Total_Hospital_IG_Loc_Govts",
  "Own_Hospital_Total_Exp",
  "Hosp_Other_Total_Exp",
  "Total_Highways_Tot_Exp",
  "Total_Highways_Dir_Exp",
  "Total_Highways_Cap_Out",
  "Total_Highways_Current_Exp",
  "Total_Highways_Construct",
  "Regular_Hwy_Total_Exp",
  "Toll_Hwy_Total_Expend",
  "Transit_Sub_Total_Exp",
  "Hous___Com_Total_Exp",
  "Libraries_Total_Expend",
  "Natural_Res_Total_Exp",
  "Parking_Total_Expend",
  "Parks___Rec_Total_Exp",
  "Police_Prot_Total_Exp",
  "Prot_Insp_Total_Exp",
  "Public_Welf_Total_Exp",
  "Welf_Categ_Total_Exp",
  "Welf_Cash_Total_Exp",
  "Welf_Ins_Total_Exp",
  "Welf_NEC_Total_Expend",
  "Sewerage_Total_Expend",
  "SW_Mgmt_Total_Expend",
  "Water_Trans_Total_Exp",
  "General_NEC_Total_Exp",
  "Total_Util_Total_Exp",
  "Total_Util_Inter_Exp",
  "Total_Util_Cap_Outlay",
  "Total_Util_Current_Exp",
  "Total_Util_Construct",
  "Water_Util_Total_Exp",
  "Elec_Util_Total_Exp",
  "Gas_Util_Total_Exp",
  "Trans_Util_Total_Exp",
  "Emp_Ret_Total_Expend",
  "Unemp_Comp_Total_Exp",
  "Total_Debt_Outstanding",
  "Total_Long_Term_Debt_Out",
  "Total_Beg_LTD_Out",
  "Total_LTD_Issued",
  "Total_LTD_Iss_FFC",
  "Total_LTD_Iss_NG",
  "Total_LTD_Retired",
  "Total_LTD_Ret_FFC",
  "Total_LTD_Ret_NG",
  "Total_LTD_Out",
  "Total_LTD_Out_Utility",
  "Total_LTD_Out_FFC",
  "Total_Cash___Securities",
  "Emp_Retire_Total_Sec"
  ]
supported_attributes = [key.lower() for key in supported_attributes]
len(supported_attributes)

107

In [0]:
ask_lohith = [key for key in supported_attributes if key not in good_attributes]
len(ask_lohith)

33

In [0]:
missing[ask_lohith].sort_values()

total_hospital_construct       0.111058
air_trans_total_expend         0.137274
total_beg_ltd_out              0.205434
welf_ins_total_exp             0.273117
own_hospital_total_exp         0.292183
total_ltd_ret_ng               0.296473
welf_cash_total_exp            0.312202
total_ltd_iss_ng               0.314109
sewerage_total_expend          0.374643
sw_mgmt_total_expend           0.407531
total_ltd_out_ffc              0.408484
total_ltd_ret_ffc              0.408961
toll_hwy_total_expend          0.475214
total_util_total_exp           0.479504
total_util_current_exp         0.487131
total_ltd_iss_ffc              0.514299
water_trans_total_exp          0.525739
total_util_cap_outlay          0.594376
transit_sub_total_exp          0.606768
total_hospital_ig_loc_govts    0.648713
total_utility_revenue          0.648713
hosp_other_total_exp           0.661582
trans_util_total_exp           0.681602
total_util_construct           0.683031
water_util_total_exp           0.769781
